<a href="https://colab.research.google.com/github/agarwalpratik/aiml/blob/main/BoostingSmoteClassifierSocial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
#XGBoost is not a part of Sklearn package

import xgboost
import numpy as np
import pandas as pd

In [23]:
data = pd.read_csv('Social_Network_Ads.csv')

In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   Age              400 non-null    int64
 1   EstimatedSalary  400 non-null    int64
 2   Purchased        400 non-null    int64
dtypes: int64(3)
memory usage: 9.5 KB


In [25]:
data.head()

,Age,EstimatedSalary,Purchased
0,19,19000,0
1,35,20000,0
2,26,43000,0
3,27,57000,0
4,19,76000,0


In [26]:
data.Purchased.value_counts()

,count
Purchased,
0,257
1,143


In [38]:
# Separate the features and target
X = data.drop(columns=['Purchased'])
y = data['Purchased']




In [39]:
# Apply SMOTE
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=1)
X_resampled, y_resampled = smote.fit_resample(X, y)


In [40]:

# Combine the resampled features and target into a single dataframe
resampled_data = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.DataFrame(y_resampled, columns=['Purchased'])], axis=1)



In [41]:
# Display the resampled class distribution
print(resampled_data['Purchased'].value_counts())



Purchased
0    257
1    257
Name: count, dtype: int64


In [42]:
features = resampled_data.iloc[:,0:2].values
label = resampled_data.iloc[:,2].values


In [43]:
features

array([[    19,  19000],
       [    35,  20000],
       [    26,  43000],
       ...,
       [    53,  26000],
       [    55, 132863],
       [    48,  95986]])

In [53]:
CL = 0.90
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

maxTestScore, maxTrainScore, finalRs, finalMd = 0,0,0,0

for i in range(1,101):
  X_train,X_test,y_train,y_test = train_test_split(features,label,test_size=0.2,random_state=i)
  for md in range(3,20):
    for lr in (0.001,0.002,0.003,0.004,0.005):
      model = XGBClassifier(max_depth=md,learning_rate=lr)
      model.fit(X_train,y_train)
      testScore = model.score(X_test,y_test)
      trainScore = model.score(X_train,y_train)

      if testScore > trainScore: # and testScore > CL:
        if testScore > maxTestScore:
          maxTestScore = testScore
          maxTrainScore = trainScore
          finalRs = i
          finalMd = md

print(f"Test Score : {maxTestScore} TrainScore : {maxTrainScore} RS: {finalRs} Max_Depth: {finalMd}")

Test Score : 0.9805825242718447 TrainScore : 0.9124087591240876 RS: 58 Max_Depth: 3


In [54]:
CL = 0.94
from xgboost import XGBRFClassifier
from sklearn.model_selection import train_test_split

maxTestScore, maxTrainScore, finalRs, finalMd = 0,0,0,0

for i in range(1,101):
  X_train,X_test,y_train,y_test = train_test_split(features,label,test_size=0.2,random_state=i)
  for md in range(3,20):
    for lr in (0.001,0.002,0.003,0.004,0.005):
      model = XGBRFClassifier(max_depth=md,learning_rate=lr)
      model.fit(X_train,y_train)
      testScore = model.score(X_test,y_test)
      trainScore = model.score(X_train,y_train)

      if testScore > trainScore: # and testScore > CL:
        if testScore > maxTestScore:
          maxTestScore = testScore
          maxTrainScore = trainScore
          finalRs = i
          finalMd = md

print(f"Test Score : {maxTestScore} TrainScore : {maxTrainScore} RS: {finalRs} Max_Depth: {finalMd}")

Test Score : 0.9514563106796117 TrainScore : 0.9148418491484185 RS: 34 Max_Depth: 3
